In [2]:
import faiss
import numpy as np
from tools.utils.settings import DefaultPath as defpath

import pandas as pd
from tools.utils.utils import create_token_set
from tools.utils.mongodb_utils import get_mongodb_collections, get_one_document_from_mongodb_by_key
from tools.embedding import FastTextTableEmbedder

In [3]:
tabenc = FastTextTableEmbedder('/data4/nanni/tesi-magistrale/models/fasttext/cc.en.300.bin')

In [4]:
mongoclient, collections = get_mongodb_collections('wikipedia', 'standard')

In [5]:
dataset = 'wikipedia'
test_dir = f'{defpath.data_path.tests}/a_test'
index_path = f'{test_dir}/wikipedia/embedding/col_idx_mfasttext.index'
index = faiss.read_index(index_path)
blacklist = {'{"$numberDouble": "NaN"}', 'comment', 'story'} if dataset == 'gittables' else set()

In [21]:
from statistics import mean
from itertools import groupby

def compute_results(xq_ids, D, I, k):
    ids = np.unique(xq_ids)

    rI = np.concatenate((xq_ids, I), axis=1)
    rI = np.split(rI[:, 1:], np.unique(rI[:, 0], return_index=True)[1][1:])

    rD = np.concatenate((xq_ids, D), axis=1)
    rD = np.split(rD[:, 1:], np.unique(rD[:, 0], return_index=True)[1][1:])
    print(rI)
    print(rD)
    print('rI:', len(rI), rI[0].shape, len(query_ids), xq_ids.shape)
    print('rD:', len(rD), rD[0].shape, len(query_ids), xq_ids.shape)
    print(np.unique(xq_ids))

    return [
            [
                int(y[0]) for y in
                sorted([[g[0], mean(x[1] for x in g[1])] for g in groupby(raw_group, key=lambda x: x[0])], key=lambda x: x[1])[:k]
            ]
            for raw_group in 
                [
                    sorted(
                        [
                            x for j in range(rI[i].shape[0]) for x in zip(rI[i][j], rD[i][j]) if x[0] not in {ids[i], -1}
                        ], 
                        key=lambda x: x[0]) 
                    for i in range(len(rI))
                ]
        ]

from tqdm import tqdm
d = 300
xq, xq_ids = np.empty(shape=(0, d), dtype=np.float64), np.empty(shape=(0, 1), dtype=np.int32)
batch_size = 0
results = []
# query_ids = range(980, 100)
query_ids = [55]
k = 10
# batch query processing, because it's significantly faster
for qid in tqdm(query_ids, total=len(query_ids), disable=True):
    # when reached the batch threshold, execute the search for the 
    # current batch 
    doc = get_one_document_from_mongodb_by_key('_id_numeric', int(qid), *collections)
    colemb = tabenc.embedding_table(doc['content'], doc['numeric_columns'], blacklist)
    ids = np.expand_dims(np.repeat([qid], colemb.shape[0]), axis=0)
    xq_ids = np.concatenate((xq_ids, ids.T))
    xq = np.concatenate((xq, colemb), axis=0)
    
    if xq.shape[0] > batch_size:
        D, I = index.search(xq, int(k))
        print(xq.shape[0], xq_ids.shape[0], D.shape, I.shape)
        batch_results = compute_results(xq_ids, D, I, k)
        
        for j, jqid in enumerate(np.unique(xq_ids)):
            results.append((jqid, batch_results[j], []))

        xq, xq_ids = np.empty(shape=(0, d), dtype=np.float64), np.empty(shape=(0, 1), dtype=np.int32)
    


4 4 (4, 10) (4, 10)
[array([[     55, 2662654, 2663531, 2572926, 2577169, 2573285, 2658131,
        2638439, 1668581, 1596085],
       [     55,      56,   16446,     104,  982073,  982102,  474681,
         168415,  168424,      43],
       [      4,       8,      10,      13,      43,      55,      74,
             75,      81,      84],
       [     55,  183505,      79,  238987,  278499,  453288,  593808,
         849060,  849803,   36439]])]
[array([[0.00000000e+00, 1.86056551e-02, 1.97069198e-02, 2.06442233e-02,
        2.27631107e-02, 2.29005609e-02, 2.32388992e-02, 2.37023942e-02,
        2.38051321e-02, 2.39079613e-02],
       [0.00000000e+00, 1.43143982e-02, 1.52567066e-02, 1.54692223e-02,
        2.14616116e-02, 2.14616135e-02, 2.16705482e-02, 2.16865521e-02,
        2.16865521e-02, 2.20784545e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000

In [31]:
doc55 = get_one_document_from_mongodb_by_key('_id_numeric', 55, *collections)
doc = get_one_document_from_mongodb_by_key('_id_numeric', 5, *collections)

In [32]:
pd.DataFrame(doc55['content'])

,0,1,2,3
0,Hans Woellke,Germany,,Q
1,Sulo Bärlund,Finland,,Q
2,Gerhard Stöck,Germany,,Q
3,Sam Francis,United States,,Q
4,Jack Torrance,United States,,Q
5,Dimitri Zaitz,United States,,Q
6,František Douda,Czechoslovakia,,Q
7,Arnold Viiding,Estonia,,Q
8,Gunnar Bergh,Sweden,,Q
9,Hans-Heinrich Sievert,Germany,,Q


In [33]:
pd.DataFrame(doc['content'])

,0,1,2
0,1996,Bottle Rocket,Kumar
1,1998,Rushmore,Mr. Littlejeans
2,2001,The Royal Tenenbaums,Pagoda
3,2002,Bomb the System,Kumar Baba
4,2003,Duplex,Indian Restaurant Owner (voice only)
5,2004,The Terminal,Gupta Rajan
6,2005,Romance & Cigarettes,Da Da Kumar
7,2005,RevoLOUtion,Kumar
8,2006,10 Items or Less,Lee
9,2007,The Darjeeling Limited,Old man on train


In [34]:
emb55 = tabenc.embedding_table(doc55['content'], doc55['numeric_columns'])
emb = tabenc.embedding_table(doc['content'], doc['numeric_columns'])

In [43]:
np.linalg.norm(emb55, axis=1)

array([0.49362156, 0.70070606, 0.        , 0.99999994], dtype=float32)

In [60]:
from tools.sloth.utils import parse_table

table = doc55['content']

table = [column for i, column in enumerate(parse_table(table, len(table[0]), 0)) if doc55['numeric_columns'][i] == 0 and any(column)]    

In [64]:
np.array([
    tabenc.model.get_sentence_vector('')
    for i in range(0)
]).shape

(0,)

In [61]:
len(table)

3

In [62]:
table[2]

['Q',
 'Q',
 'Q',
 'Q',
 'Q',
 'Q',
 'Q',
 'Q',
 'Q',
 'Q',
 'Q',
 'Q',
 'Q',
 'Q',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [44]:
emb55[2]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.